In [ ]:
from typing import Any, Protocol, runtime_checkable

## Messages

From RFC-000: Eventually, we will move away from dictionaries and represent all messages with objects (dataclasses or Pydantic base models). For now, we will use existing implementation.

In [ ]:
Message = dict[str, Any]

## Configuration

Eventually, we will move away from dictionaries and represent the configuration with objects, but for now we will use the existing dict implementation.

In [ ]:
Configuration = dict[str, Any]

## Client messages

We need to define a message protocol that all the clients will align to, we are using OpenAIs ChatCompletion at the moment

In [ ]:
from autogen.oai.oai_models import ChatCompletion

## Client protocol

The basic client protocol

In [ ]:
@runtime_checkable
class ClientProtocol(Protocol):
    def create(self, history: Message) -> ChatCompletion:
        """
        Generates a response/message based on the conversation history.
        """
        pass

    def accept(cls, config: Configuration) -> tuple[bool, int]:
        """
        Returns a tuple:
        - A boolean indicating if the client matches.
        - An integer representing the specificity (higher is more specific).
        """
        pass

## Client Factory

In [ ]:
class ClientFactory:
    _registry: list[ClientProtocol] = []

    @classmethod
    def register(cls, client_class: ClientProtocol) -> ClientProtocol:
        """
        Decorator to register a new client class.
        """
        cls._registry.append(client_class)
        return client_class

    @classmethod
    def create_client(cls, config: Configuration) -> ClientProtocol:
        # Collect matches and their priorities
        matches = []
        for client_class in cls._registry:
            is_match, priority = client_class.accept(config)
            if is_match:
                matches.append((priority, client_class))

        if not matches:
            raise ValueError(f"No compatible client found for configuration: {config}")

        # Sort by priority (highest priority first)
        matches.sort(key=lambda x: x[0], reverse=True)

        # Log if there are multiple matches
        if len(matches) > 1:
            print(f"Warning: Multiple clients matched. Using {matches[0][1].__name__} with priority {matches[0][0]}.")

        # Instantiate the highest-priority client
        return matches[0][1](config)

## Example client

In [ ]:
@ClientFactory.register
class ExampleClient:
    priority = 0

    def __init__(self, config: Configuration):
        self.name = "ExampleClient"

    def create(self, history: Message) -> ChatCompletion:
        return "This is a generated response from ExampleClient." # Should be chat completion object, str now before we have the model

    @classmethod
    def accept(self, config: Configuration) -> bool:
        return config.get("type") == "example", self.priority

In [ ]:
# Configuration for the client
config = {"type": "example"}

# Create the client using the factory
client = ClientFactory.create_client(config)

# Generate a response
history = [{"role": "user", "content": "Hello!"}]
response = client.create(history)

print(f"Client: {client.name}")
print(f"Response: {response}")

## Structured output, function calling etc..

The functionaltities that each client supports will be different. For example, one client might support structured output, while another might support function calling. The factory pattern allows us to create a client instance based on the provided configuration. The client instance can then be used to generate a response based on the conversation history.

Existing clients can continue using the simpler accept method with a default priority of 0.

This way, we can support previous implementations of clients with the lowest priority and slowly cannibalise them into more specific, higher priority clients.

In [ ]:
# Example clients
@ClientFactory.register
class DefaultOpenAIClient:
    def __init__(self, config: Configuration):
        self.name = "DefaultOpenAIClient"

    def create(self, history: Message) -> ChatCompletion:
        return "This is a response from DefaultOpenAIClient."

    @classmethod
    def accept(cls, config: Configuration) -> tuple[bool, int]:
        is_match = config.get("api_type") == "openai"
        priority = 1  # General OpenAI client has lower specificity
        return is_match, priority


@ClientFactory.register
class OpenAIStructuredClient:
    def __init__(self, config: Configuration):
        self.name = "OpenAIStructuredClient"

    def create(self, history: Message) -> ChatCompletion:
        return """{"response": "This is a response from OpenAIStructuredClient."}"""

    @classmethod
    def accept(cls, config: Configuration) -> tuple[bool, int]:
        is_match = config.get("api_type") == "openai" and "structured_output" in config
        priority = 10  # Structured client has higher specificity
        return is_match, priority

In [ ]:
# Configurations
general_config = {"api_type": "openai"}
structured_config = {"api_type": "openai", "structured_output": True}

# Create general OpenAI client
general_client = ClientFactory.create_client(general_config)
print(f"Client: {general_client.name}")
print(f"Response: {general_client.create([])}")

# Create structured OpenAI client
structured_client = ClientFactory.create_client(structured_config)
print(f"Client: {structured_client.name}")
print(f"Response: {structured_client.create([])}")